In [1]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Extract text from both PDFs
pdf1_text = extract_text_from_pdf(r"C:\Users\yashswi shukla\Desktop\Project\ChatBot\pdfs\1_FinancialResults_05022025142214.pdf")
pdf2_text = extract_text_from_pdf(r"C:\Users\yashswi shukla\Desktop\Project\ChatBot\pdfs\Amaar raja Earnings Summary.pdf")

# Save extracted text for labeling
with open("financial_data.txt", "w", encoding="utf-8") as f:
    f.write(pdf1_text + "\n\n" + pdf2_text)


In [ ]:
import json
import torch
from torch.utils.data import Dataset
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments, pipeline
from sklearn.model_selection import train_test_split

# ✅ Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Load Pretrained FinBERT Model
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Fix classifier mismatch issue & Move Model to GPU
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=5, ignore_mismatched_sizes=True
).to(device)

# ✅ Define labels
labels_to_ids = {
    "COMPANY": 0,
    "REPORT_DATE": 1,
    "PROFIT_BEFORE_TAX": 2,
    "REVENUE": 3,
    "NET_PROFIT": 4
}
ids_to_labels = {v: k for k, v in labels_to_ids.items()}

# ✅ Load JSON Manually
with open(r"C:\Users\yashswi shukla\Desktop\Project\ChatBot\try3\datasets\load_dataset\labeled_financial_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# ✅ Split Data: 80% Train, 20% Eval
train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)


# ✅ Custom Dataset Class
class FinancialNERDataset(Dataset):
    def __init__(self, data, tokenizer, labels_to_ids, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.labels_to_ids = labels_to_ids
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]["text"]
        entities = self.data[idx]["entities"]

        # ✅ Tokenize text while keeping track of token start/end positions
        tokenized_inputs = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_offsets_mapping=True,
            return_tensors="pt"
        )

        offset_mapping = tokenized_inputs.pop("offset_mapping")[0]  # Remove batch dim
        labels = torch.zeros(self.max_length, dtype=torch.long)  # Initialize label tensor

        # ✅ Correctly Assign Entity Labels
        for start, end, label in entities:
            for i, (token_start, token_end) in enumerate(offset_mapping):
                if token_start is None or token_end is None:  # Ignore special tokens ([CLS], [SEP])
                    continue
                if token_start >= start and token_end <= end:  
                    labels[i] = self.labels_to_ids[label]  # ✅ Assign correct label

        tokenized_inputs["labels"] = labels.to(device)  # ✅ Move labels to GPU
        return {key: val.squeeze(0).to(device) for key, val in tokenized_inputs.items()}  # ✅ Move inputs to GPU


# ✅ Create Dataset Objects
train_dataset = FinancialNERDataset(train_data, tokenizer, labels_to_ids)
eval_dataset = FinancialNERDataset(eval_data, tokenizer, labels_to_ids)

# ✅ Training Arguments (Ensure GPU is Used)
training_args = TrainingArguments(
    output_dir="./finbert_finetuned",
    evaluation_strategy="epoch",  # ✅ Enable evaluation after each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=50,  # ✅ Increased for better performance
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,  # ✅ Enable mixed precision for faster training on GPU
    save_total_limit=2  # ✅ Prevents excessive model checkpoint storage
)

# ✅ Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # ✅ Provide evaluation dataset
    tokenizer=tokenizer
)

# ✅ Start Fine-Tuning
trainer.train()

# ✅ Save Fine-Tuned Model
model.save_pretrained("./finbert_finetuned")
tokenizer.save_pretrained("./finbert_finetuned")


# ✅ **Perform Inference (AFTER Fine-Tuning)**
print("\n🚀 Running Inference on New Text...")

# Load fine-tuned model
ner_pipeline = pipeline("ner", model="./finbert_finetuned", tokenizer="./finbert_finetuned", aggregation_strategy="simple", device=0)  # ✅ Use GPU (device=0)

# Sample input
text = "Reliance Industries Ltd. announced its financial results on 10th June 2025. Profit before tax was ₹312.7 crores."

# Predict entities
entities = ner_pipeline(text)

# ✅ Convert numerical labels (LABEL_0, LABEL_1, etc.) to actual names (COMPANY, REPORT_DATE, etc.)
for entity in entities:
    entity["entity_group"] = ids_to_labels.get(int(entity["entity_group"].replace("LABEL_", "")), "UNKNOWN")

print("🔍 Extracted Entities:", entities)


Using device: cuda


Some weights of BertForTokenClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\ProgramData\miniconda3\envs\pytorch_env\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\yashswi shukla\AppData\Local\Temp\ipykernel_19224\2864407038.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Device set to use cuda:0


[{'entity_group': 'LABEL_0', 'score': np.float32(0.83339566), 'word': 'reliance industries ltd. announced its financial results on 10th june 2025. profit before tax was ₹312. 7 crores.', 'start': 0, 'end': 112}]


In [11]:
import torch
import pdfplumber
from transformers import pipeline, AutoTokenizer
import os

# ✅ Load the fine-tuned model and tokenizer
model_path = "./finbert_finetuned"
ner_pipeline = pipeline("ner", model=model_path, tokenizer=model_path, aggregation_strategy="simple")

# ✅ Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text.strip() if text else None  # Return None if no text found

# ✅ Function to predict entities in a PDF
def predict_entities_from_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    if not text:
        print(f"⚠️ No text found in {pdf_path}")
        return None
    
    # Run model inference
    entities = ner_pipeline(text)

    # Convert labels (e.g., LABEL_0 -> COMPANY, REPORT_DATE, etc.)
    ids_to_labels = {
        "LABEL_0": "COMPANY",
        "LABEL_1": "REPORT_DATE",
        "LABEL_2": "PROFIT_BEFORE_TAX",
        "LABEL_3": "REVENUE",
        "LABEL_4": "NET_PROFIT"
    }
    for entity in entities:
        entity["entity_group"] = ids_to_labels.get(entity["entity_group"], "UNKNOWN")
    
    return entities

# ✅ Specify PDF files (replace with actual file paths)
pdf_files = [r"C:\Users\yashswi shukla\Desktop\Project\ChatBot\pdfs\1_FinancialResults_05022025142214.pdf", r"C:\Users\yashswi shukla\Desktop\Project\ChatBot\pdfs\Amaar raja Earnings Summary.pdf"]

# ✅ Process each PDF and display results
for pdf in pdf_files:
    print(f"\n📄 Processing: {pdf}")
    extracted_entities = predict_entities_from_pdf(pdf)
    if extracted_entities:
        print("🔍 Extracted Entities:", extracted_entities)


Device set to use cuda:0



📄 Processing: C:\Users\yashswi shukla\Desktop\Project\ChatBot\pdfs\1_FinancialResults_05022025142214.pdf
🔍 Extracted Entities: [{'entity_group': 'COMPANY', 'score': np.float32(0.92366385), 'word': "evereaidnyd ustriinedsil at d. evere ady regoo. f fi : c2 e, r ainpeayrk ko, l k - a7t0a0 019 cnl : l 31402wb1934plc007993 givem e power, givem ered dat5the f : e br2u0a2r5y thnea tisotnoaeclxk c haonfg bes el imited thcea lcsuttoetcxakc hange indlitad phirjoezeej eebhloiym ited exchapnlgacez - a1b,, l ogc, k towedrasls, at lr eet7 l, yornasn ge bandkruarc loam plex mumb - a4i00 00 1 kolk - a7t00a00i band ( rema ) u, m b - a4i00 05 1 [ sccroidp5e 3 : 1 508 [ ] s ccroidp0e 0 : 0 029 ] [ symebvoelr : e ady ] deasri / rm sadam, subd : i sclousnudrerere gula3t0oi fos ne b ( il istoibnlgi gaatniddo inssc losure requirermeegnutlsa ) 2t 0i1 - oo5nu st, c oomfbe o amrede thienlogdn 5t hf ebruary 2025 purstuoar netg ul3a0to ifso enb ( il isotbilnigg aatniddoi nssc lroesquurier ements ), regula2t0i1 